## Create and install VPN client configuration
Point-to-Site connections use certificates to authenticate. This article shows how to create a self-signed root certificate and generate client certificates using PowerShell on Windows 10 or Windows Server 2016. If needs are for different certificate instructions, see [Certificates - Linux or Certificates - MakeCert](https://docs.microsoft.com/en-us/azure/vpn-gateway/vpn-gateway-certificates-point-to-site-linux).

The steps performed in this article are on a computer running Windows 10 or Windows Server 2016. The PowerShell cmdlets that used to generate certificates are part of the operating system and do not work on other versions of Windows. The Windows 10 or Windows Server 2016 computer is only needed to generate the certificates. Once the certificates are generated, it can be uploaded or installed on any supported client operating system.

In [ ]:
$RG = ""                 # Name of intended Resource Group
$GWName = ""             # VPN Gateway name
$P2SRootCertName = ""    # Name of Root Certificate. For example 'P2SRootCert2.cer'
$filePathForCert = ""    # Path of  the certificate file. For example 'D:\Downloads\rootcert2.cer'

### **Create a self-signed root certificate**
_New-SelfSignedCertificate_ cmdlet is used to create a self-signed root certificate. For additional parameter information, see [New-SelfSignedCertificate](https://technet.microsoft.com/itpro/powershell/windows/pkiclient/new-selfsignedcertificate).

Use the following example to create the self-signed root certificate. The following example creates a self-signed root certificate named 'P2SRootCert' that is automatically installed in 'Certificates-Current User\Personal\Certificates'. certificate can be viewed by opening certmgr.msc, or Manage User Certificates.

In [ ]:
$cert = New-SelfSignedCertificate -Type Custom -KeySpec Signature `
-Subject "CN=P2SRootCert" -KeyExportPolicy Exportable `
-HashAlgorithm sha256 -KeyLength 2048 `
-CertStoreLocation "Cert:\CurrentUser\My" -KeyUsageProperty Sign -KeyUsage CertSign

### **Generate a client certificate**
Each client computer that connects to a VNet using Point-to-Site must have a client certificate installed. A client certificate can be generated from the self-signed root certificate, and then it can export and install the client certificate. If the client certificate is not installed, authentication fails.
This example continues from the previous section and uses the declared '$cert' variable.

Modify and run the example to generate a client certificate. If the following example is executed without modifying it, the result is a client certificate named 'P2SChildCert'. If name of the child certificate needs to be something else,then modify the CN value. Do not change the TextExtension when running this example. The client certificate which is generated is automatically installed in 'Certificates - Current User\Personal\Certificates' on computer.

In [ ]:
New-SelfSignedCertificate -Type Custom -DnsName P2SChildCert -KeySpec Signature `
-Subject "CN=P2SChildCert" -KeyExportPolicy Exportable `
-HashAlgorithm sha256 -KeyLength 2048 `
-CertStoreLocation "Cert:\CurrentUser\My" `
-Signer $cert -TextExtension @("2.5.29.37={text}1.3.6.1.5.5.7.3.2")

### **Replace the file path variable with file path to the exported root certificate, and then run the variable cmdlets:**

In [ ]:
$cert = new-object System.Security.Cryptography.X509Certificates.X509Certificate2($filePathForCert)
$CertBase64_3 = [system.convert]::ToBase64String($cert.RawData)
$p2srootcert = New-AzVpnClientRootCertificate -Name $P2SRootCertName -PublicCertData $CertBase64_3

### **Upload the public key information to Azure**
_New-AzVpnClientRootCertificate_ cmdlet creates a new VPN client root certificate

In [ ]:
 New-AzVpnClientRootCertificate -VpnClientRootCertificateName $P2SRootCertName `
    -VirtualNetworkGatewayName $GWName -ResourceGroupName $RG `
    -PublicCertData $CertBase64_3

### **Verify that the root certificate uploaded**
_Get-AzVpnClientRootCertificate_ cmdlet gets information about VPN root certificates

In [ ]:
Get-AzVpnClientRootCertificate -ResourceGroupName $RG -VirtualNetworkGatewayName $GWName